In [1]:
#피봇pivot테이블 : 데이터 열 중 2개의 열을 각각 행, 열 인덱스로 데이터 펼쳐놓는 것
#첫번째 인수는 행 인덱스, 두번째 인수는 열 인덱스, 마지막은 데이터로 사용할 열이름
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [4]:
df1.pivot("도시", "연도", "인구")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [21]:
df1.set_index(["도시", "연도"])[["인구"]].unstack()
#원래 set_index(["A","B"])이렇게하면 다중인덱스인데이걸 .unstack()으로 품

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

In [22]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [28]:
##행, 열 인덱스 조건을 만족하는 데이터가 2개 이상이면 에러 발생
#이 경우에서는 (지역, 연도) = (수도권, 2015)가 2개 이상이라 에러발생
try:
    df1.pivot("지역", "연도", "인구")
except ValueError as e:
    print("Error:", e)

Error: Index contains duplicate entries, cannot reshape


In [29]:
#그룹 분석 : 키가 지정하는 조건에 맞는 데이터가 2개 이상
#groupby method : 피봇테이블과 달리 키에 의해 결정된 데이터가 여러개일 경우 미리 지정한 연산으로 대표값을 계산
#1.시리즈나 데이터프레임에서 groupby 메서드 호출해 그룹화
#2.그룹 객체에 대해 그룹연산 수행
#groupby 메서드는 데이터를 그룹 별로 분류하는 역할
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [30]:
#key1의 값에 따른 data1의 평균은?
groups = df2.groupby(df2.key1) # 그룹A 그룹B로 구분한 그룹 데이터 생성
groups

In [32]:
groups.groups #GroupBy 클래스 객체에는 각 그룹 데이터의 인덱스 저장한 groups 속성


{'A': Int64Index([0, 1, 4], dtype='int64'),
 'B': Int64Index([2, 3], dtype='int64')}

In [57]:
#연습 문제
groups.sum().describe()

,data1,data2
count,2.000000,2.000000
mean,7.500000,75.000000
std,0.707107,7.071068
min,7.000000,70.000000
25%,7.250000,72.500000
50%,7.500000,75.000000
75%,7.750000,77.500000
max,8.000000,80.000000


In [37]:
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [38]:
df2.groupby(df2.key1)["data1"].sum() 

key1
A    8
B    7
Name: data1, dtype: int64

In [43]:
groups = df2.groupby(df2.key1)
groups.sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [45]:
df2.data1.groupby([df2.key1, df2.key2]).sum() #키가 복수면 리스트 사용

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

In [48]:
df2.data1.groupby([df2.key1, df2.key2]).sum().unstack("key2")

key2,one,two
key1,,
A,6,2
B,3,4


In [49]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [51]:
df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack("연도")

연도,2005,2010,2015
지역,,,
경상권,3512547,3393191,3448737
수도권,9762546,9894685,12794763


In [85]:
import seaborn as sns
iris = sns.load_dataset("iris")

In [59]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio) #연산 직접 만든 후 agg메서드 사용

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


In [60]:
#describe 사용하면 다양한 기술 통계 값 한번에 구함
iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

In [65]:
#apply사용시 데이터프레임을만들 수 있음
def top3_petal_length(df):
    return df.sort_values(by = "petal_length", ascending = False)[:3]
#데이터프레임에서 sort_values 사용하려면 by인수로 정렬기준 열 지정
iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

In [72]:
#transform method는 그룹별 대푯값이 아니라 그룹별 계산해 데이터프레임 자체를 변화
def q3cut(s):
    return pd.qcut(s, 3, labels = ["소", "중", "대"])
#qcut : 구간별로 동일 갯수로 나눔
iris["petal_length_class"] = iris.groupby(iris.species)["petal_length"].transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

,petal_length,petal_length_class
140,5.6,중
141,5.1,소
142,5.1,소
143,5.9,대
144,5.7,중
145,5.2,소
146,5.0,소
147,5.2,소
148,5.4,중
149,5.1,소


In [75]:
#연습 문제 2 : 종별로 꽃잎길이, 꽃잎폭 등의 평균 구하라
def average(x):
    return x.sum() / 50

iris.groupby(iris.species).agg(average)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [76]:
#pivot_table : pivot과 groupbby의 중간 성격
#groupby처럼 그룹분석, 최종적으로 pivot처럼 피봇테이블 만듬
#pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')
#data: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
#values: 분석할 데이터프레임에서 분석할 열
#index: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
#columns: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
#aggfunc: 분석 메서드
#fill_value: NaN 대체 값
#margins: 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부
#margins_name: 마진 열(행)의 이름

In [77]:
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [78]:
#메서드라 data가 별도로 입력되지 않았음
df1.pivot_table("인구", "도시", "연도")

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [81]:
#margins=True시 aggfunc(default = mean)로 주어진 분석 방법을 모든 열에 적용
df1.pivot_table("인구", "도시", "연도",margins = True, margins_name = "합계")

연도,2005,2010,2015,합계
도시,,,,
부산,3512547.0,3393191.0,3448737.0,3.451492e+06
서울,9762546.0,9631482.0,9904312.0,9.766113e+06
인천,NaN,263203.0,2890451.0,1.576827e+06
합계,6637546.5,4429292.0,5414500.0,5.350809e+06


In [83]:
#행 또는 열 인덱스에 리스트 넣으면 다중 인덱스 테이블 만듬
df1.pivot_table("인구", index = ["연도", "도시"])

인구
연도   도시         
2005 부산  3512547
     서울  9762546
2010 부산  3393191
     서울  9631482
     인천   263203
2015 부산  3448737
     서울  9904312
     인천  2890451

In [89]:
#TIP 데이터 예제
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [90]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [91]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


In [92]:
#우선 성별로 나누어 데이터 갯수 세어보기
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [93]:
tips.groupby("sex").size()

sex
Male      157
Female     87
dtype: int64

In [96]:
tips.groupby(["sex","smoker"]).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

In [100]:
tips.pivot_table("tip_pct", "sex", "smoker", aggfunc = "count", margins = True)

smoker,Yes,No,All
sex,,,
Male,60,97,157
Female,33,54,87
All,93,151,244


In [102]:
#성별, 흡연 여부에 따른 평균 팁 비율 살펴보기
tips.groupby("sex")[["tip_pct"]].mean()

,tip_pct
sex,
Male,0.157651
Female,0.166491


In [103]:
tips.groupby("smoker")[["tip_pct"]].mean()

,tip_pct
smoker,
Yes,0.163196
No,0.159328


In [105]:
#pivot_table 명령도 사용 가능
tips.pivot_table("tip_pct", ["sex", "smoker"])

tip_pct
sex    smoker          
Male   Yes     0.152771
       No      0.160669
Female Yes     0.182150
       No      0.156921

In [108]:
tips.groupby("sex")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
sex                                                                          
Male     157.0  0.157651  0.064778  0.035638  0.121389  0.153492  0.186240   
Female    87.0  0.166491  0.053632  0.056433  0.140416  0.155581  0.194266   

                  
             max  
sex               
Male    0.710345  
Female  0.416667

In [109]:
tips.groupby("smoker")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
smoker                                                                       
Yes       93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   
No       151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   

                  
             max  
smoker            
Yes     0.710345  
No      0.291990

In [110]:
tips.groupby(["smoker", "sex"])[["tip_pct"]].describe()

tip_pct                                                    \
                count      mean       std       min       25%       50%   
smoker sex                                                                
Yes    Male      60.0  0.152771  0.090588  0.035638  0.101845  0.141015   
       Female    33.0  0.182150  0.071595  0.056433  0.152439  0.173913   
No     Male      97.0  0.160669  0.041849  0.071804  0.131810  0.157604   
       Female    54.0  0.156921  0.036421  0.056797  0.139708  0.149691   

                                   
                    75%       max  
smoker sex                         
Yes    Male    0.191697  0.710345  
       Female  0.198216  0.416667  
No     Male    0.186220  0.291990  
       Female  0.181630  0.252672

In [111]:
tips.groupby("day")[["tip_pct"]].describe()

tip_pct                                                              \
       count      mean       std       min       25%       50%       75%   
day                                                                        
Thur    62.0  0.161276  0.038652  0.072961  0.138210  0.153846  0.192687   
Fri     19.0  0.169913  0.047665  0.103555  0.133739  0.155625  0.196637   
Sat     87.0  0.153152  0.051293  0.035638  0.123863  0.151832  0.188271   
Sun     76.0  0.166897  0.084739  0.059447  0.119982  0.161103  0.187889   

                
           max  
day             
Thur  0.266312  
Fri   0.263480  
Sat   0.325733  
Sun   0.710345

In [112]:
tips.groupby("time")[["tip_pct"]].describe()

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
time                                                                         
Lunch     68.0  0.164128  0.040242  0.072961  0.139147  0.154084  0.193917   
Dinner   176.0  0.159518  0.067477  0.035638  0.123192  0.155400  0.188209   

                  
             max  
time              
Lunch   0.266312  
Dinner  0.710345

In [114]:
#연습 문제 3
tips.groupby(["size","time","day"])[["tip_pct"]].describe()

tip_pct                                                    \
                   count      mean       std       min       25%       50%   
size time   day                                                              
1    Lunch  Thur     1.0  0.181728       NaN  0.181728  0.181728  0.181728   
            Fri      1.0  0.223776       NaN  0.223776  0.223776  0.223776   
     Dinner Sat      2.0  0.231832  0.132796  0.137931  0.184882  0.231832   
2    Lunch  Thur    47.0  0.164024  0.037705  0.078616  0.139802  0.153846   
            Fri      5.0  0.181969  0.052856  0.117735  0.153657  0.180921   
     Dinner Thur     1.0  0.159744       NaN  0.159744  0.159744  0.159744   
            Fri     11.0  0.162659  0.047408  0.103555  0.133739  0.146628   
            Sat     53.0  0.155289  0.050392  0.035638  0.129422  0.155219   
            Sun     39.0  0.180870  0.108245  0.059447  0.121057  0.166297   
3    Lunch  Thur     4.0  0.144599  0.070665  0.072961  0.089888  0.145822   
            Fri      1.0  0.187735       NaN  0.187735  0.187735  0.187735   
     Dinner Sat     18.0  0.151439  0.044294  0.056433  0.128767  0.153479   
            Sun     15.0  0.152662  0.043604  0.071804  0.132653  0.160542   
4    Lunch  Thur     5.0  0.145515  0.031241  0.115982  0.121262  0.147059   
     Dinner Fri      1.0  0.117750       NaN  0.117750  0.117750  0.117750   
            Sat     13.0  0.138289  0.046107  0.077459  0.102522  0.136240   
            Sun     18.0  0.153168  0.043921  0.081466  0.126415  0.149691   
5    Lunch  Thur     1.0  0.121389       NaN  0.121389  0.121389  0.121389   
     Dinner Sat      1.0  0.106572       NaN  0.106572  0.106572  0.106572   
            Sun      3.0  0.159839  0.088650  0.065660  0.118927  0.172194   
6    Lunch  Thur     3.0  0.173706  0.028857  0.140940  0.162891  0.184843   
     Dinner Sun      1.0  0.103799       NaN  0.103799  0.103799  0.103799   

                                      
                       75%       max  
size time   day                       
1    Lunch  Thur  0.181728  0.181728  
            Fri   0.223776  0.223776  
     Dinner Sat   0.278782  0.325733  
2    Lunch  Thur  0.191950  0.266312  
            Fri   0.198216  0.259314  
     Dinner Thur  0.159744  0.159744  
            Fri   0.184486  0.263480  
            Sat   0.188395  0.291990  
            Sun   0.202918  0.710345  
3    Lunch  Thur  0.200533  0.213789  
            Fri   0.187735  0.187735  
     Dinner Sat   0.179938  0.230742  
            Sun   0.170312  0.227747  
4    Lunch  Thur  0.148435  0.194837  
     Dinner Fri   0.117750  0.117750  
            Sat   0.186220  0.205577  
            Sun   0.169696  0.280535  
5    Lunch  Thur  0.121389  0.121389  
     Dinner Sat   0.106572  0.106572  
            Sun   0.206928  0.241663  
6    Lunch  Thur  0.190089  0.195335  
     Dinner Sun   0.103799  0.103799

In [121]:
def peak_to_peak(x):
    return x.max() - x.min()

tips.groupby(["sex","smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

In [125]:
#여러가지 그룹여산 동시에 하기
tips.groupby(["sex","smoker"]).agg(["mean", peak_to_peak])[["total_bill"]]

total_bill             
                    mean peak_to_peak
sex    smoker                        
Male   Yes     22.284500        43.56
       No      19.791237        40.82
Female Yes     17.977879        41.23
       No      18.105185        28.58

In [126]:
#데이터 열마다 다른 연산 하고 싶다? 열라벨, 연산이름을 딕셔너리로
tips.groupby(["sex","smoker"]).agg(
{'tip_pct' : 'mean', 'total_bill' : peak_to_peak})

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

In [129]:
#pivot_table 명령으로 더 복잡한 분석
tips.pivot_table(['tip_pct', 'size'], ['sex', 'day'], 'smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

In [135]:
tips.pivot_table('size',['time','sex','smoker'], 'day',
                aggfunc = 'sum', fill_value = 0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43

In [136]:
#연습 문제 4 : 1. qcut 명령으로 3개의 나이 그룹을 만든다.
titanic = sns.load_dataset("titanic")

In [137]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [152]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["청년", "중년", "장년"])


titanic["age_class"] = titanic.groupby(titanic.pclass)["age"].transform(q3cut)
titanic[["age", "age_class"]]

,age,age_class
0,22.0,중년
1,38.0,중년
2,26.0,중년
3,35.0,중년
4,35.0,장년
...,...,...
886,27.0,중년
887,19.0,청년
888,NaN,NaN
889,26.0,청년


In [162]:
#연습 문제 3 : 2 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산
titanic['suv_pct'] = titanic['survived'] / 1
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,age_class,suv_pct
886,0,2,male,27.0,0,0,13.00,S,Second,man,True,NaN,Southampton,no,True,중년,0.0
887,1,1,female,19.0,0,0,30.00,S,First,woman,False,B,Southampton,yes,True,청년,1.0
888,0,3,female,NaN,1,2,23.45,S,Third,woman,False,NaN,Southampton,no,False,NaN,0.0
889,1,1,male,26.0,0,0,30.00,C,First,man,True,C,Cherbourg,yes,True,청년,1.0
890,0,3,male,32.0,0,0,7.75,Q,Third,man,True,NaN,Queenstown,no,True,장년,0.0


In [182]:
#연습 문제 3-2
df4 = titanic.pivot_table('suv_pct',['sex', 'pclass'],['age'],fill_value=0)

In [192]:
#연습 문제 3 : 3 성별 및 선실에 의한 생존율을 피봇 데이터 형태
titanic.pivot_table('suv_pct',"sex", "pclass")

pclass,1,2,3
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447
